##  Saved dataset locally with name HZZ.root from "http://scikit-hep.org/uproot/examples/HZZ.root"

In [1]:
import numpy as np
import uproot
events=uproot.open("HZZ.root")["events"]

In [2]:
columns=events.arrays(["*Muon*"])

## Description of Columns

In [3]:
print "Shape of the available data"
for key,value in columns.items():
    print key, np.array(columns[key]).shape

Shape of the available data
NMuon (2421,)
Muon_Charge (3825,)
Muon_E (3825,)
Muon_Iso (3825,)
Muon_Pz (3825,)
Muon_Py (3825,)
Muon_Px (3825,)


In [4]:
Muon_Px = columns["Muon_Px"].content
Muon_Py = columns["Muon_Py"].content
Muon_Pz = columns["Muon_Pz"].content
Muon_E= columns["Muon_E"].content
NMuon= columns["NMuon"]

# 3 functions defined
<b><p>1. calP_E( )</p>
<p>2. numMuons( )</p>
<p>3. getPossibleZMass( )</p></b>
<hr>
<p><b>1. calP_E( )</b>: func to be used in a vectorized call for getting Momemtum and Energy of all 3825 Muons in 2421 events.

In [5]:
from vectorized import vectorize
def calP_E(index, Muon_Px, Muon_Py, Muon_Pz, Muon_E, totalp,totale):
    px2 = Muon_Px[index]**2
    py2 = Muon_Py[index]**2
    pz2 = Muon_Pz[index]**2
    e2 = Muon_E[index]**2
    totalp[index] = np.sqrt(px2+py2+pz2)
    totale[index] = np.sqrt(e2)

In [6]:
totalp=np.empty(len(Muon_Px))  #totalp, momemtum of all 3825 muons
totale=np.empty(len(Muon_Px))  #totale, energy of all 3825 muons
print "(shape)totalp :",totalp.shape
print "(shape)toatle :",totale.shape

(shape)totalp : (3825,)
(shape)toatle : (3825,)


In [7]:
############### Vectorized call ################
vectorize(calP_E,len(Muon_E),Muon_Px,Muon_Py,Muon_Pz,Muon_E,totalp,totale)

leading step 0 (100.0% at leading): 
    px2 = (Muon_Px[index] ** 2)
    ...advancing 1

leading step 1 (100.0% at leading): 
    py2 = (Muon_Py[index] ** 2)
    ...advancing 2

leading step 2 (100.0% at leading): 
    pz2 = (Muon_Pz[index] ** 2)
    ...advancing 3

leading step 3 (100.0% at leading): 
    e2 = (Muon_E[index] ** 2)
    ...advancing 4

leading step 4 (100.0% at leading): 
    totalp[index] = np.sqrt(((px2 + py2) + pz2))
    ...advancing 5

leading step 5 (100.0% at leading): 
    totale[index] = np.sqrt(e2)
    ...advancing 6



7

<p><b>2. numMuons( ):</b> func returning the index of muons out of 3825 Muons, condition to that each event doesn't have atleast 2 Muons.

In [8]:
def numMuons(start,stop):
    notpossible_Zboson=[]
    for i in range(len(start)):
        if ( stop[i]-start[i]<2):
            notpossible_Zboson.append(start[i])
    return notpossible_Zboson

In [9]:
start=columns["Muon_E"].starts
stop=columns["Muon_E"].stops
notpossible_Zboson=numMuons(start,stop)

In [10]:
#### index of muons from 3825 Muons which are unlikely to be Z Boson
print "No. of Muons to be discarded: ",len(notpossible_Zboson)
print "Index are:",notpossible_Zboson

No. of Muons to be discarded:  1008
Index are: [2, 13, 22, 23, 30, 42, 43, 46, 47, 54, 59, 62, 63, 64, 73, 74, 74, 76, 83, 86, 89, 90, 91, 98, 103, 108, 113, 118, 120, 121, 126, 127, 132, 139, 139, 140, 149, 150, 151, 154, 155, 156, 159, 166, 172, 173, 178, 179, 180, 199, 206, 211, 214, 217, 218, 221, 226, 235, 240, 241, 248, 249, 258, 263, 264, 273, 305, 308, 310, 313, 314, 319, 320, 327, 328, 331, 334, 334, 337, 338, 339, 344, 345, 354, 366, 369, 372, 373, 378, 379, 386, 392, 392, 393, 398, 409, 410, 415, 416, 419, 420, 441, 446, 447, 455, 456, 457, 458, 459, 465, 466, 469, 470, 473, 474, 478, 481, 482, 483, 486, 489, 490, 491, 498, 499, 500, 501, 502, 508, 509, 512, 515, 516, 517, 522, 523, 524, 531, 533, 536, 547, 548, 549, 556, 557, 562, 565, 566, 567, 578, 579, 584, 587, 592, 593, 593, 598, 601, 602, 617, 618, 619, 620, 627, 628, 629, 634, 641, 644, 647, 650, 652, 655, 656, 663, 666, 667, 668, 669, 670, 673, 674, 679, 688, 689, 692, 697, 702, 703, 722, 723, 724, 731, 732, 733, 73

<p><b>3. getPossibleZMass( ):</b> Vectorized func to be called so as to get mass of all possible Z Bosons.</p>
<br>
Here, "totale" contains the energy of all muons which had been in an event with 2 muons . Similiarly "totalm" contains all mass of muons which had been in event with 2 muons.

In [11]:
def getPossibleZMass(index,totale,totalp,totalm):
    p2=totalp[index]**2
    e2=totale[index]**2
    totalm[index]=np.sqrt(np.absolute(e2-p2))*1000 #Assumption mass to be multiplied with 1000 to get unit in "Gev"

In [12]:
#### remove thoes muon "momemtum" and "energy"  which are unlikely to be a decay product of Z Boson
totalp=np.delete(totalp,notpossible_Zboson)
totale=np.delete(totale,notpossible_Zboson)
print "After filtering shape of totalp and totale :",totalp.shape,totale.shape

After filtering shape of totalp and totale : (2839,) (2839,)


# Problem Set to calculate mass of muons of possible Z Bosons

In [13]:
totalm=np.empty(len(totale))
################ Vectorized call ######################
vectorize(getPossibleZMass,len(totale),totale,totalp,totalm)

leading step 0 (100.0% at leading): 
    p2 = (totalp[index] ** 2)
    ...advancing 1

leading step 1 (100.0% at leading): 
    e2 = (totale[index] ** 2)
    ...advancing 2

leading step 2 (100.0% at leading): 
    totalm[index] = (np.sqrt(np.absolute((e2 - p2))) * 1000)
    ...advancing 3



4

## Return mass of Z boson with around ±1 tolerance of 91Gev

In [14]:
notbest=[]
for i in range(len(totalm)):
    if not((totalm[i]-91 >= -1 )and (totalm[i]-91 <= 1)):
        notbest.append(i)
print np.delete(totalm,notbest)

[90.26714996 91.44529293 90.89767816 91.37391805 91.84836151 91.97384277
 91.88857868 91.83295305 91.11483711 91.2099798  90.04649756 91.60284199
 91.70939427]
